### This is the Notebook for Lecure 24 - Cryptography

In this lecture, we will review
<ol>
    <li>Basic Cryptographic Oprtations: <b>Caesar Cipher</b></li>
</ol>

In [14]:
import string

def caesar_cipher(message, key=13):
    
    ''' Encode/decode message using  rotational shift '''

    result = ""
    # transverse the plain text
    for i in range(len(message)):
        char = message[i]
   
        # Encrypt uppercase characters in plain text
        if (char.isupper()):
    
            result += chr((ord(char) + key-65) % 26 + 65)
   
        # Encrypt lowercase characters in plain text
        else:
            result += chr((ord(char) + key - 97) % 26 + 97)
      
    return result


def caesar_cipher_test( message, key= 13 ):
    
    print ("Plain Text : " + message)
    print ("Cipher     : " + caesar_cipher(message, key) )

In [15]:
caesar_cipher_test( "Cheer, Cheer for Old Notre Dame!", 8 )

Plain Text : Cheer Cheer for Old Notre Dame
Cipher     : KpmmzvKpmmzvnwzvWtlvVwbzmvLium


In [17]:
caesar_cipher_test( "Cheer, Cheer for Old Notre Dame!", 13 )

Plain Text : Cheer, Cheer for Old Notre Dame!
Cipher     : PurremaPurreasbeaByqaAbgeraQnzrb
